In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
#圖片像素
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] 

In [2]:
attrs = ["amount", "freq", "avg_interval"]

In [3]:
raw_data = pd.read_csv("2022AIGO_H_LAB測試資料_雲端行動科技.csv")
raw_data.dropna(axis=0, subset=["item_tag"], inplace=True)

In [4]:
data_2020 = raw_data[raw_data["datetime"].str.startswith("2020")]
data_2020.loc[:, "datetime"] = pd.to_datetime(data_2020["datetime"], format="%Y-%m-%d")
data_2020["month"] = data_2020["datetime"].dt.month

C:\Users\Sylvia\anaconda3\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)
<ipython-input-4-bb98c96d1407>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2020["month"] = data_2020["datetime"].dt.month


In [5]:
all_user = (data_2020.user_id.unique())

In [6]:
# df_2020 = data_2020[data_2020["item_tag"] != "優惠活動/折扣/集點"]
df_2020 = data_2020[data_2020["item_tag"] != "無法分類"]
df_2020 = df_2020[df_2020["item_tag"] != "餐飲需求"]

In [7]:
#先不要用avg_interval
df_group_alltime = df_2020.groupby(["user_id", "item_tag"])[["amount", "datetime", "inv_num"]].agg(
    amount = ("amount", "sum"),
    freq = ("inv_num", lambda x : len(x.unique())),
    avg_interval = ("datetime", lambda x : x.sort_values().diff().sum().days / len(x))
)

In [23]:
df_ouput_alltime = pd.DataFrame(index = all_user, columns=[f'{i}_{j}'  for i in df_2020.item_tag.unique() for j in attrs])
def make_output_matrix(x):
    id = x.index.get_level_values("user_id")[0]
    x = x.droplevel((0))
    df_temp = x.stack()
    df_temp.index = [f"{x}_{y}" for x,y in df_temp.index]
    df_temp = df_temp.to_frame("").T
    df_ouput_alltime.loc[id, df_temp.columns] = df_temp.values

df_group_alltime.groupby(level = (0)).apply(make_output_matrix)

""


In [24]:
df_ouput_alltime.loc[:, df_ouput_alltime.columns.str.endswith("avg_interval")] = df_ouput_alltime.loc[:, df_ouput_alltime.columns.str.endswith("avg_interval")].replace([0, np.nan],1000)


In [25]:
df_ouput_alltime.fillna(0, inplace= True)

In [26]:
df_ouput_alltime = df_ouput_alltime.abs()

In [28]:
#MinMaxScaler() 在MinMaxScaler中是給定了一個明確的最大值與最小值。每個特徵中的最小值變成了0，最大值變成了1。數據會縮放到到[0,1]之間。
#fit_transform() fit_transform():fit_transform(partData)是先對partData作fit()的功能，找到該partData的整體統計特性之指標，如平均值、標準差、最大最小值等等(能依據不同目的套用這些指標在不同的轉換(即後面的transform()動作)上，再實行transform(partData)以對partData進行標準化(英文稱為normalization, 主要有兩種方法: min-max normalization or standard deviation normalization)或歸一化等動作.
from sklearn.preprocessing import MinMaxScaler
minmax_sc = MinMaxScaler()
df_ouput_alltime_scaled = minmax_sc.fit_transform(df_ouput_alltime)

In [27]:
#暫時不用avg_time
df_ouput_alltime = df_ouput_alltime[df_ouput_alltime.columns[~df_ouput_alltime.columns.str.endswith('avg_interval')]]

# No scaled

In [14]:
#V=W*H
#alpha : Constant that multiplies the regularization terms. Set it to zero to have no regularization.
#l1_ratio : The regularization mixing parameter, with 0 <= l1_ratio <= 1. For l1_ratio = 0 the penalty is an elementwise L2 penalty (aka Frobenius Norm). For l1_ratio = 1 it is an elementwise L1 penalty. For 0 < l1_ratio < 1, the penalty is a combination of L1 and L2.
'''K= 5,10,20,50 alpha= .4, l1_ratio = .5'''
from sklearn.decomposition import NMF
K = 10
nmf = NMF(n_components= K, beta_loss= 'frobenius', init='random',alpha_H = .4, l1_ratio = 1) #nndsvd更適用於sparse factorization
W = nmf.fit_transform(df_ouput_alltime)
H = nmf.components_    

In [15]:
df_H = pd.DataFrame(H, columns=df_ouput_alltime.columns)
df_H

,乳製風味飲品_amount,乳製風味飲品_freq,綠茶_amount,綠茶_freq,停車費_amount,停車費_freq,碳酸飲料_amount,碳酸飲料_freq,牙刷_amount,牙刷_freq,...,其他文具圖書_amount,其他文具圖書_freq,點鈔機_amount,點鈔機_freq,投影機_amount,投影機_freq,美甲玩具_amount,美甲玩具_freq,美髮剪_amount,美髮剪_freq
0,0.049642,0.000000,0.116333,0.000000,0.075282,0.000000,0.290711,0.006085,0.065903,0.000000,...,0.0,0.0,0.000000,0.0,0.000798,0.0,0.002250,0.0,0.000000,0.0
1,2.411677,0.047928,4.719436,0.071448,0.000000,0.000000,4.353882,0.091460,0.297285,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
2,3.109837,0.035602,3.708242,0.050697,0.096614,0.000000,4.487333,0.042583,0.363126,0.003281,...,0.0,0.0,0.001691,0.0,0.039604,0.0,0.005802,0.0,0.000465,0.0
3,14.133159,0.222545,18.310357,0.368532,6.597557,0.149665,7.889686,0.208166,1.814342,0.016547,...,0.0,0.0,0.004217,0.0,0.061759,0.0,0.028749,0.0,0.000000,0.0
4,0.000000,0.000000,8.324490,0.118299,2.282743,0.000000,0.000000,0.000000,1.322106,0.000000,...,0.0,0.0,0.006110,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
5,2.594644,0.039880,1.387603,0.000000,3.099273,0.017623,0.995427,0.016414,1.354040,0.002597,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
6,2.773406,0.049300,7.278860,0.037786,2.179959,0.000000,5.537904,0.014268,4.534551,0.004113,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
7,4.572298,0.138800,1.188184,0.016946,1.392313,0.000000,0.772143,0.013319,0.058991,0.000000,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0
8,2.238149,0.028181,8.228335,0.203597,9.287668,0.241219,4.058390,0.087486,1.471060,0.010896,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.012892,0.0,0.000000,0.0
9,16.893846,0.311833,38.760760,1.173327,15.609827,0.088857,16.860528,0.393025,2.056626,0.008075,...,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.001560,0.0


In [131]:
for i in range(K):
    display(pd.DataFrame(pd.concat([df_H.loc[i, :].T.nlargest(5), df_H[df_H>0].loc[i, :].T.nsmallest(5)])))

,0
店家名稱_amount,8.415544e+02
優惠活動/折扣/集點_amount,3.736909e+02
其他服飾配件_amount,3.441721e+02
其他料理_amount,9.436890e+01
運動飲料_amount,8.529613e+01
藍芽接收/發射器_freq,5.050824e-07
內接式硬碟_freq,1.808065e-06
保健箱/急救包_freq,3.735929e-06
棒球/壘球_freq,4.056544e-06
碎紙機_freq,4.132634e-06


,1
汽油_amount,3.341954e+02
其他飾品_amount,3.249298e+02
其他料理_amount,2.699646e+02
智慧型手機_amount,2.247171e+02
香菸_amount,1.101811e+02
燕窩_freq,1.485246e-07
簡報筆_freq,4.012282e-06
高跟鞋_freq,4.463432e-06
風水/命理/占星書_freq,4.480317e-06
其他袋子_freq,8.580953e-06


,2
手續/服務/外送費_amount,7.311530e+02
運費_amount,5.686237e+02
筆記型電腦_amount,5.721249e+01
優惠活動/折扣/集點_amount,1.359481e+01
乳清蛋白_amount,7.453641e+00
行李秤/手提秤_freq,2.742388e-08
彩妝刷具_freq,2.038443e-07
保健箱/急救包_freq,3.213935e-07
兒童潤膚_freq,3.258222e-07
電腦記憶體_freq,3.424542e-07


,3
襪子_amount,6.237898e+02
口腔特殊護理/清潔_amount,4.040334e+02
卸妝油/乳/水_amount,4.025044e+02
優惠活動/折扣/集點_amount,3.359812e+02
妝前乳/隔離霜_amount,3.259702e+02
電鑽/電動起子_freq,9.534111e-07
其他肉類罐頭_freq,1.091221e-06
高跟鞋_freq,1.696022e-06
吸塵器周邊_freq,1.830766e-06
電子門鎖_freq,2.259420e-06


,4
電費_amount,1.066385e+03
其他料理_amount,1.415806e+02
現沖咖啡_amount,1.777654e+01
儲值現金/點數_amount,1.483989e+01
水費_amount,1.284062e+01
其他茶具用品_freq,2.172466e-08
棒球/壘球_freq,4.924432e-07
充電電池/充電器_freq,1.213025e-06
交換器_freq,1.527543e-06
其他文具圖書_freq,1.655368e-06


In [132]:
for i in range(K):
    display(pd.DataFrame(pd.concat([df_H.loc[i, df_H.columns.str.endswith("amount")].T.nlargest(5), df_H[df_H>0].loc[i, df_H.columns.str.endswith("amount")].T.nsmallest(5)])))
    display(pd.DataFrame(pd.concat([df_H.loc[i, df_H.columns.str.endswith("freq")].T.nlargest(5), df_H[df_H>0].loc[i, df_H.columns.str.endswith("freq")].T.nsmallest(5)])))
    #display(pd.DataFrame(pd.concat([df_H.loc[i, df_H.columns.str.endswith("interval")].T.nlargest(5), df_H[df_H>0].loc[i, df_H.columns.str.endswith("interval")].T.nsmallest(5)])))

,0
店家名稱_amount,841.554407
優惠活動/折扣/集點_amount,373.690947
其他服飾配件_amount,344.172096
其他料理_amount,94.368898
運動飲料_amount,85.296130
其他肉類罐頭_amount,0.000020
其他文具圖書_amount,0.000158
保健箱/急救包_amount,0.000189
祭祀用品_amount,0.000236
快煮壺/電熱水壺/瓶周邊_amount,0.000430


,0
優惠活動/折扣/集點_freq,1.062536e+00
麵包/漢堡/蛋糕_freq,3.890907e-01
其他料理_freq,3.672264e-01
手續/服務/外送費_freq,3.355354e-01
店家名稱_freq,3.259993e-01
藍芽接收/發射器_freq,5.050824e-07
內接式硬碟_freq,1.808065e-06
保健箱/急救包_freq,3.735929e-06
棒球/壘球_freq,4.056544e-06
碎紙機_freq,4.132634e-06


,1
汽油_amount,334.195408
其他飾品_amount,324.929791
其他料理_amount,269.964648
智慧型手機_amount,224.717129
香菸_amount,110.181099
其他袋子_amount,0.000205
奶精_amount,0.000294
其他肉類罐頭_amount,0.000870
其他文具圖書_amount,0.000930
風水/命理/占星書_amount,0.001601


,1
優惠活動/折扣/集點_freq,2.593499e+00
麵包/漢堡/蛋糕_freq,1.089360e+00
其他料理_freq,8.788508e-01
香菸_freq,8.312081e-01
現沖咖啡_freq,8.092159e-01
燕窩_freq,1.485246e-07
簡報筆_freq,4.012282e-06
高跟鞋_freq,4.463432e-06
風水/命理/占星書_freq,4.480317e-06
其他袋子_freq,8.580953e-06


,2
手續/服務/外送費_amount,731.153040
運費_amount,568.623668
筆記型電腦_amount,57.212490
優惠活動/折扣/集點_amount,13.594814
乳清蛋白_amount,7.453641
其他茶具用品_amount,0.000006
棒球/壘球_amount,0.000007
其他袋子_amount,0.000009
美髮剪_amount,0.000011
其他肉類罐頭_amount,0.000018


,2
優惠活動/折扣/集點_freq,9.760668e-01
運費_freq,8.920808e-01
手續/服務/外送費_freq,7.761150e-01
麵包/漢堡/蛋糕_freq,1.785055e-01
瓶/罐裝多多_freq,6.124336e-02
行李秤/手提秤_freq,2.742388e-08
彩妝刷具_freq,2.038443e-07
保健箱/急救包_freq,3.213935e-07
兒童潤膚_freq,3.258222e-07
電腦記憶體_freq,3.424542e-07


,3
襪子_amount,623.789847
口腔特殊護理/清潔_amount,404.033379
卸妝油/乳/水_amount,402.504406
優惠活動/折扣/集點_amount,335.981198
妝前乳/隔離霜_amount,325.970185
其他袋子_amount,0.000007
其他茶具用品_amount,0.000132
奶精_amount,0.000153
其他肉類罐頭_amount,0.000259
藥盒_amount,0.000272


,3
優惠活動/折扣/集點_freq,1.171643e+00
汽油_freq,5.749580e-01
儲值現金/點數_freq,5.268808e-01
襪子_freq,3.822416e-01
妝前乳/隔離霜_freq,2.047795e-01
電鑽/電動起子_freq,9.534111e-07
其他肉類罐頭_freq,1.091221e-06
高跟鞋_freq,1.696022e-06
吸塵器周邊_freq,1.830766e-06
電子門鎖_freq,2.259420e-06


,4
電費_amount,1066.385089
其他料理_amount,141.580588
現沖咖啡_amount,17.776536
儲值現金/點數_amount,14.839893
水費_amount,12.840622
其他茶具用品_amount,0.000031
其他文具圖書_amount,0.000056
其他美髮周邊_amount,0.000102
棒球/壘球_amount,0.000134
料理夾_amount,0.000161


,4
優惠活動/折扣/集點_freq,4.208598e-01
其他料理_freq,3.855312e-01
手續/服務/外送費_freq,3.326638e-01
現沖咖啡_freq,2.385019e-01
麵包/漢堡/蛋糕_freq,1.334213e-01
其他茶具用品_freq,2.172466e-08
棒球/壘球_freq,4.924432e-07
充電電池/充電器_freq,1.213025e-06
交換器_freq,1.527543e-06
其他文具圖書_freq,1.655368e-06


# Scaled

In [31]:
K = 5
nmf_sc = NMF(n_components= K, beta_loss= 'frobenius', init='random',alpha_H = .4, l1_ratio = 1, max_iter = 1000)
W_sc = nmf_sc.fit_transform(df_ouput_alltime_scaled)
H_sc = nmf_sc.components_

In [38]:
K = 5
nmf_sc = NMF(n_components= K, solver= 'mu', beta_loss= 'kullback-leibler', init='random',alpha_H = .4, l1_ratio = 1, max_iter = 1000)
W_sc = nmf_sc.fit_transform(df_ouput_alltime_scaled)
H_sc = nmf_sc.components_

In [39]:
df_H_sc = pd.DataFrame(H_sc, columns=df_ouput_alltime.columns)
df_H_sc

,乳製風味飲品_amount,乳製風味飲品_freq,綠茶_amount,綠茶_freq,停車費_amount,停車費_freq,碳酸飲料_amount,碳酸飲料_freq,牙刷_amount,牙刷_freq,...,其他文具圖書_amount,其他文具圖書_freq,點鈔機_amount,點鈔機_freq,投影機_amount,投影機_freq,美甲玩具_amount,美甲玩具_freq,美髮剪_amount,美髮剪_freq
0,0.000014,0.000012,0.000078,0.000070,0.000023,0.000032,0.000081,0.000065,0.000035,0.000035,...,0.000000,0.000000,0.00002,0.00002,0.000025,0.000039,0.00002,0.00002,1.963249e-05,1.963444e-05
1,0.000664,0.000556,0.000751,0.000831,0.000085,0.000085,0.000593,0.000496,0.000107,0.000306,...,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000e+00,0.000000e+00
2,0.000052,0.000035,0.000031,0.000012,0.000040,0.000030,0.000011,0.000009,0.000163,0.000285,...,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000e+00,0.000000e+00
3,0.000104,0.000039,0.000036,0.000004,0.000015,0.000009,0.000041,0.000010,0.000144,0.000173,...,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,2.142911e-09,1.957552e-10
4,0.000122,0.000056,0.000073,0.000042,0.000004,0.000008,0.000110,0.000049,0.000090,0.000140,...,0.000019,0.000019,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000e+00,0.000000e+00


In [40]:
for i in range(K):
    display(pd.DataFrame(pd.concat([df_H_sc.loc[i, :].T.nlargest(5), df_H_sc[df_H_sc>0].loc[i, :].T.nsmallest(5)])))

,0
雲端空間_freq,3.861479e-04
桌子_freq,3.615668e-04
行車記錄器/導航_freq,3.438086e-04
螢幕/顯示器_freq,2.957519e-04
其他傢俱_freq,2.945021e-04
吸管_amount,2.621898e-16
簽字筆/奇異筆/麥克筆_amount,2.676427e-16
熱狗/香腸_amount,3.289736e-16
粥_freq,3.949215e-16
其他寵物用品_freq,4.267416e-16


,1
優惠活動/折扣/集點_freq,1.207576e-03
麵包/漢堡/蛋糕_freq,1.194627e-03
雞料理_amount,1.012030e-03
紅茶_amount,8.578445e-04
綠茶_freq,8.306462e-04
香水_amount,2.783453e-16
童書_freq,2.875004e-16
菜心類罐頭_freq,4.390016e-16
烘焙粉_amount,8.090177e-16
四物精華_amount,1.557536e-15


,2
T恤_freq,4.636321e-04
底妝_amount,4.383547e-04
防曬乳/噴霧_freq,4.362704e-04
吹風機_freq,4.229495e-04
眼線液/筆/膠_freq,4.088668e-04
積木/樂高_freq,2.822344e-16
讀卡機_freq,3.645725e-16
其他素食類罐頭_freq,4.175190e-16
海苔_freq,4.727213e-16
音源傳輸/轉換_freq,7.701680e-16


,3
維他命/礦物質_freq,3.622469e-04
浴廁清潔/疏通劑_freq,3.417033e-04
牙膏/牙粉_freq,3.148599e-04
水費_freq,3.112168e-04
兒童紙尿褲_amount,2.953711e-04
瑜珈_freq,5.003086e-16
智能手錶/手環周邊_freq,5.255111e-16
皮帶/腰帶_amount,8.495189e-16
夜燈_amount,1.014967e-15
冷麵_amount,1.028242e-15


,4
植物油_freq,5.437314e-04
豆皮/豆腐_freq,4.853667e-04
料理塊/包/罐_freq,4.747293e-04
餐具清潔劑_freq,4.672026e-04
沾拌醬_amount,4.231705e-04
燴飯_freq,3.875672e-16
乳酸飲料_amount,1.826701e-15
舒緩貼布_amount,1.975962e-15
馬桶刷/通便器_amount,2.005376e-15
其他鞋類_amount,2.409592e-15


In [41]:
for i in range(K):
    display(pd.DataFrame(pd.concat([df_H_sc.loc[i, df_H_sc.columns.str.endswith("amount")].T.nlargest(5), df_H_sc[df_H_sc>0].loc[i, df_H_sc.columns.str.endswith("amount")].T.nsmallest(5)])))
    display(pd.DataFrame(pd.concat([df_H_sc.loc[i, df_H_sc.columns.str.endswith("freq")].T.nlargest(5), df_H_sc[df_H_sc>0].loc[i, df_H_sc.columns.str.endswith("freq")].T.nsmallest(5)])))
    display(pd.DataFrame(pd.concat([df_H_sc.loc[i, df_H_sc.columns.str.endswith("interval")].T.nlargest(5), df_H_sc[df_H_sc>0].loc[i, df_H_sc.columns.str.endswith("interval")].T.nsmallest(5)])))

,0
雨具用品_amount,2.246362e-04
插座擴增/延長線_amount,2.145824e-04
筆記型電腦_amount,1.695902e-04
電信/網路費_amount,1.670470e-04
手機/平板/筆電充電傳輸設備_amount,1.532774e-04
吸管_amount,2.621898e-16
簽字筆/奇異筆/麥克筆_amount,2.676427e-16
熱狗/香腸_amount,3.289736e-16
料理夾_amount,5.820099e-16
墨鏡_amount,1.386720e-15


,0
雲端空間_freq,3.861479e-04
桌子_freq,3.615668e-04
行車記錄器/導航_freq,3.438086e-04
螢幕/顯示器_freq,2.957519e-04
其他傢俱_freq,2.945021e-04
粥_freq,3.949215e-16
其他寵物用品_freq,4.267416e-16
兒童潤膚_freq,5.859052e-16
蛋餅/抓餅/蔥油餅_freq,5.872174e-16
其他生鮮_freq,1.106694e-15


,0


,1
雞料理_amount,1.012030e-03
紅茶_amount,8.578445e-04
壽司/飯糰_amount,7.581608e-04
綠茶_amount,7.507912e-04
其他飯食_amount,7.387134e-04
香水_amount,2.783453e-16
烘焙粉_amount,8.090177e-16
四物精華_amount,1.557536e-15
紙巾_amount,2.722079e-15
油漆/塗料_amount,2.910609e-15


,1
優惠活動/折扣/集點_freq,1.207576e-03
麵包/漢堡/蛋糕_freq,1.194627e-03
綠茶_freq,8.306462e-04
其他飯食_freq,7.978513e-04
熱狗/香腸_freq,7.821028e-04
童書_freq,2.875004e-16
菜心類罐頭_freq,4.390016e-16
痘痘貼_freq,1.769092e-15
水龍頭用品_freq,2.853256e-15
精華液/油/乳/霜_freq,2.947970e-15


...

,3
維他命/礦物質_freq,3.622469e-04
浴廁清潔/疏通劑_freq,3.417033e-04
牙膏/牙粉_freq,3.148599e-04
水費_freq,3.112168e-04
其他除蟲用品_freq,2.890802e-04
瑜珈_freq,5.003086e-16
智能手錶/手環周邊_freq,5.255111e-16
拉力/彈力帶_freq,1.146525e-15
袖套_freq,1.671387e-15
鐵蛋_freq,1.929586e-15


,3


,4
沾拌醬_amount,4.231705e-04
火鍋料_amount,3.895775e-04
泡菜_amount,3.371577e-04
生麵條_amount,3.344533e-04
生豬肉_amount,2.975629e-04
乳酸飲料_amount,1.826701e-15
舒緩貼布_amount,1.975962e-15
馬桶刷/通便器_amount,2.005376e-15
其他鞋類_amount,2.409592e-15
睡衣/居家服_amount,7.807819e-15


,4
植物油_freq,5.437314e-04
豆皮/豆腐_freq,4.853667e-04
料理塊/包/罐_freq,4.747293e-04
餐具清潔劑_freq,4.672026e-04
生麵條_freq,3.986843e-04
燴飯_freq,3.875672e-16
抗菌液/噴霧_freq,2.954524e-15
眼部保養_freq,3.090640e-15
其他舒壓好眠_freq,7.735818e-15
掃地機器人周邊_freq,1.184151e-14


,4


In [33]:
K = 10
nmf_sc = NMF(n_components= 10, init='random', random_state=0, max_iter = 1000)
W_sc = nmf_sc.fit_transform(df_ouput_alltime_scaled)
H_sc = nmf_sc.components_

In [ ]:
K = 10
nmf_sc = NMF(n_components= 10, solver= 'mu', beta_loss= 'kullback-leibler', init='random', random_state=0, max_iter = 1000)
W_sc = nmf_sc.fit_transform(df_ouput_alltime_scaled)
H_sc = nmf_sc.components_

In [34]:
df_H_sc = pd.DataFrame(H_sc, columns=df_ouput_alltime.columns)
df_H_sc

,乳製風味飲品_amount,乳製風味飲品_freq,綠茶_amount,綠茶_freq,停車費_amount,停車費_freq,碳酸飲料_amount,碳酸飲料_freq,牙刷_amount,牙刷_freq,...,其他文具圖書_amount,其他文具圖書_freq,點鈔機_amount,點鈔機_freq,投影機_amount,投影機_freq,美甲玩具_amount,美甲玩具_freq,美髮剪_amount,美髮剪_freq
0,0.027235,0.025112,0.015058,0.000000,0.000000,0.000053,0.014036,0.008860,0.009826,0.016685,...,0.000000,0.000000,0.000645,0.000645,0.001296,0.002462,0.000000,0.000000,0.000000,0.000000
1,0.049049,0.023606,0.000000,0.000000,0.005551,0.000000,0.027231,0.007499,0.060683,0.041877,...,0.000000,0.000000,0.000000,0.000000,0.001326,0.001064,0.000000,0.000000,0.000000,0.000000
2,0.056430,0.042909,0.052665,0.082148,0.072899,0.121678,0.100743,0.078295,0.131061,0.252720,...,0.002770,0.002770,0.004539,0.004539,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.086787,0.040491,0.064174,0.000000,0.041568,0.039407,0.000000,0.000000,0.198192,0.521062,...,0.000000,0.000000,0.000000,0.000000,0.000062,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.091389,0.000000,0.093393,0.000000,0.004306,0.000000,0.131140,0.000000,0.216578,0.245603,...,0.028953,0.028953,0.000000,0.000000,0.000000,0.000000,0.031123,0.031123,0.013273,0.013273
5,0.023606,0.020545,0.133212,0.034096,0.012690,0.000000,0.000000,0.000000,0.215026,0.090174,...,0.000000,0.000000,0.015162,0.015162,0.002867,0.022658,0.000000,0.000000,0.020902,0.020902
6,0.533001,0.453609,0.577862,0.665440,0.051362,0.052283,0.537146,0.440539,0.037229,0.163102,...,0.005525,0.005525,0.000000,0.000000,0.004554,0.000610,0.004813,0.004813,0.000000,0.000000
7,0.013885,0.003000,0.000000,0.000000,0.002422,0.000000,0.047518,0.012523,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000687,0.002652,0.029562,0.029562,0.000000,0.000000
8,0.054267,0.002533,0.012090,0.021433,0.015852,0.015411,0.057418,0.046710,0.113735,0.165878,...,0.000000,0.000000,0.008027,0.008027,0.015014,0.026137,0.000000,0.000000,0.000000,0.000000
9,0.265753,0.164832,0.138370,0.159616,0.020487,0.020054,0.050430,0.034368,0.027508,0.049386,...,0.000000,0.000000,0.021429,0.021429,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [35]:
for i in range(K):
    display(pd.DataFrame(pd.concat([df_H_sc.loc[i, :].T.nlargest(5), df_H_sc[df_H_sc>0].loc[i, :].T.nsmallest(5)])))

,0
原子筆/原子筆芯_freq,0.205852
檔案收納_freq,0.204187
膠帶/黏貼用品_freq,0.201461
毛筆/水彩筆_freq,0.190982
皮夾/錢包_freq,0.180700
妝前乳/隔離霜_freq,0.000007
指甲油_amount,0.000017
卸妝油/乳/水_freq,0.000033
琴酒_freq,0.000035
門簾/窗簾/蚊帳_freq,0.000039


,1
電器收納_freq,0.278389
螢幕/顯示器_freq,0.263672
衣架/衣夾_freq,0.261940
行車記錄器/導航_freq,0.214611
修甲工具_freq,0.214183
酵素/乳酸/益生菌_amount,0.000051
春聯/對聯_amount,0.000063
隨身碟_freq,0.000065
生豆榖雜糧_amount,0.000070
其他女性調理_freq,0.000076


,2
履歷表_freq,0.294399
履歷表_amount,0.293436
鋁箔紙_freq,0.278642
桌上型電腦_freq,0.263672
衛生紙/面紙_freq,0.263341
氣球_freq,0.000007
涼/拖鞋_amount,0.000032
打卡鐘/紙_amount,0.000036
隱形眼鏡_amount,0.000053
錄音筆_amount,0.000083


,3
牙膏/牙粉_freq,0.779694
女性私密清潔/保養_freq,0.578265
化妝水/噴霧_freq,0.563168
防曬乳/噴霧_freq,0.554871
眼線液/筆/膠_freq,0.541356
投影機_amount,0.000062
機車車罩/護套_amount,0.000132
電扇_amount,0.000216
牙線/牙線棒_amount,0.000268
電動工具配件_amount,0.000268


,4
植物油_freq,0.967093
豆皮/豆腐_freq,0.954346
鮮蛋_amount,0.732065
堅果類零食_amount,0.725016
料理塊/包/罐_freq,0.723726
兒童防曬乳/噴霧_amount,0.000080
桌球_amount,0.000092
桌球_freq,0.000092
溫/濕度計_amount,0.000109
內接式硬碟_amount,0.000130


,5
T恤_freq,1.162382
長褲_freq,1.099415
短褲_freq,0.855301
短褲_amount,0.798527
皮夾/錢包_freq,0.778191
桌機供電設備_amount,0.000014
桌遊/益智遊戲_freq,0.000016
洗碗機周邊_freq,0.000030
麥克風_amount,0.000033
別針/胸針/徽章_freq,0.000038


,6
麵包/漢堡/蛋糕_freq,0.957722
優惠活動/折扣/集點_freq,0.953516
雞料理_amount,0.822722
綠茶_freq,0.665440
紅茶_amount,0.661896
兒童潤膚_freq,0.000081
保健箱/急救包_freq,0.000084
網路線/接頭_freq,0.000093
素香鬆_amount,0.000140
顯示器保護鏡_amount,0.000159


,7
行車記錄器/導航_freq,0.660622
製冰工具_freq,0.609924
洗衣膠囊_freq,0.567570
其他個人清潔/保養_freq,0.565701
床墊/涼墊/保暖墊_freq,0.555301
醬瓜類罐頭_freq,0.000011
行李箱/袋_amount,0.000068
衣物/棉被收納_amount,0.000091
大豆/植物異黃酮_amount,0.000109
素香鬆_freq,0.000149


,8
封口夾_freq,0.733308
餐具清潔工具_freq,0.644264
桌子_freq,0.612110
掛鉤_freq,0.545323
料理夾_freq,0.539838
卡片_freq,0.000006
弦樂器_amount,0.000018
弦樂器_freq,0.000018
磁力能量貼/戴品_freq,0.000041
乳液_amount,0.000060


,9
寫字墊/文書墊_freq,0.498373
剪刀_freq,0.472650
鉛筆盒/袋/筆筒_freq,0.469902
科普/百科_freq,0.469556
膠帶/黏貼用品_freq,0.463944
熨斗_amount,0.000002
卡式爐/瓦斯罐_freq,0.000030
冷氣_freq,0.000033
情趣用品_amount,0.000034
數位相機_amount,0.000038
